EXERCISE 4

Importing Libraries : 

In [1]:
import cv2 as cv
import numpy as np

Q - 1 : Adaptive Mean Filter

Taking Our Image as Input :

In [2]:
gaussianNoisyImage = cv.imread("GaussianNoise.jpg",0)
print(gaussianNoisyImage,"\n")
m,n = gaussianNoisyImage.shape
print(m,n)

[[172 179 167 ... 157 152 121]
 [159 165 158 ... 176 150 132]
 [166 176 152 ... 169 164 129]
 ...
 [ 42  34  46 ... 117 106 105]
 [ 38  37  50 ... 112 111 109]
 [ 47  38  52 ...  99 116  89]] 

512 512


Normalization of Image : 

In [3]:
normalizedGaussianNoisyImage = np.single(gaussianNoisyImage) / 255.0
print(normalizedGaussianNoisyImage)

[[0.6745098  0.7019608  0.654902   ... 0.6156863  0.59607846 0.4745098 ]
 [0.62352943 0.64705884 0.61960787 ... 0.6901961  0.5882353  0.5176471 ]
 [0.6509804  0.6901961  0.59607846 ... 0.6627451  0.6431373  0.5058824 ]
 ...
 [0.16470589 0.13333334 0.18039216 ... 0.45882353 0.41568628 0.4117647 ]
 [0.14901961 0.14509805 0.19607843 ... 0.4392157  0.43529412 0.42745098]
 [0.18431373 0.14901961 0.20392157 ... 0.3882353  0.45490196 0.34901962]]


Necessary Functions :

In [4]:
def levelA(piece,new_matrix,i,j,z_min,z_med,z_max,z_xy, currentPaddingSize, maxPaddingSize):
    if z_min < z_med < z_max :
        # GO TO LEVEL B.
        pixelVal = levelB(z_min,z_med,z_max,z_xy)
        return pixelVal
    else :
        currentPaddingSize +=2
        piece = take_a_piece(new_matrix,i,j,currentPaddingSize)
        z_med = np.median(piece)
        z_min = np.min(piece)
        z_max = np.max(piece)
    if currentPaddingSize < maxPaddingSize:
        # RECURSIVE CALL OF A
        levelA(piece,new_matrix,i,j,z_min,z_med,z_max,z_xy,currentPaddingSize,maxPaddingSize)
    else :
        pixelVal = z_med
        return pixelVal
    
def levelB(z_min,z_med,z_max,z_xy):
    if z_min < z_xy < z_max:
        return z_xy
    else : 
        return z_med
# Used Zero Padding on Borders.
def adaptiveMeanFilter(image,filterKernelSupportSize,varianceOfNoise):
      m,n = image.shape
      normalizedNoisyImage = normalizeImage(image)
      localVariance = 0 # sigmoid Sxy
      paddingSize = filterKernelSupportSize // 2
      big_matrix = np.zeros( (m+2*paddingSize, n+2*paddingSize) )
      big_matrix[paddingSize:paddingSize+m,paddingSize:paddingSize+n] = normalizedNoisyImage
      adaptiveMeanFilteredNormalizedImage = np.zeros( (m,n) )
      
      for i in range(paddingSize,m+paddingSize):
            for j in range(paddingSize, n + paddingSize):
                  piece = take_a_piece(big_matrix,i,j,paddingSize)
                  localAverageIntensity = np.sum(piece) / (filterKernelSupportSize**2)
                  localVariance = calculateVariance(piece)
                  noisyPixelAtXY = normalizedNoisyImage[i-paddingSize,j-paddingSize]
                  pixelValue = noisyPixelAtXY  - (varianceOfNoise / localVariance)*(noisyPixelAtXY - localAverageIntensity)
                  adaptiveMeanFilteredNormalizedImage[i-paddingSize,j-paddingSize] = pixelValue
      adaptiveMeanFilteredImage = (adaptiveMeanFilteredNormalizedImage * 255).astype(np.uint8)
      return adaptiveMeanFilteredImage

def borderReplicating(noisyimage1,paddingSize):
      first_row = noisyimage1[0,:] 
      last_row = noisyimage1[-1,:]
      first_col = noisyimage1[:,0]
      last_col = noisyimage1[:,-1]
      m,n = noisyimage1.shape
      new_matrix= np.zeros( (m + 2 * paddingSize,n + 2 * paddingSize) )
      new_matrix[paddingSize : paddingSize + m,paddingSize : paddingSize + n] = noisyimage1
      for loop_range in range(paddingSize):
            new_matrix[loop_range, paddingSize : paddingSize + m] = first_row
            new_matrix[paddingSize + m + loop_range, paddingSize : paddingSize + m] = last_row
            new_matrix[paddingSize : paddingSize + n, loop_range] = first_col
            new_matrix[paddingSize: paddingSize + n ,paddingSize + n + loop_range  ] = last_col
      for i in range(paddingSize):
            for j in range(paddingSize):
                  new_matrix[i,j] = noisyimage1[0,0]
                  new_matrix[i, j+n+paddingSize] = noisyimage1[0,511]
                  new_matrix[i+paddingSize+m, j] = noisyimage1[511,0]
                  new_matrix[i+m+paddingSize,j+n+paddingSize] = noisyimage1[511,511]
      return new_matrix

def adaptiveMedianFilter(image):
      row, col = image.shape
      # which is 3 because our max S-xy is : 7x7
      maxPaddingSize = 7 // 2
      currentPaddingSize = 3 // 2
      new_matrix = borderReplicating(image,maxPaddingSize)
      adaptiveMedianFilteredImage = np.zeros( (row,col) )
      for i in range(maxPaddingSize, maxPaddingSize+row):
            for j in range(maxPaddingSize,maxPaddingSize+col):
                  #LEVEL A
                  piece = take_a_piece(new_matrix,i,j,currentPaddingSize)
                  z_med = np.median(piece)
                  z_min = np.min(piece)
                  z_max = np.max(piece)
                  z_xy = new_matrix[i,j]
                  pixelVal = levelA(piece,new_matrix,i,j,z_min,z_med,z_max,z_xy, currentPaddingSize, maxPaddingSize)
                  adaptiveMedianFilteredImage[i-maxPaddingSize,j-maxPaddingSize] = pixelVal
      adaptiveMedianFilteredImage = np.uint8(adaptiveMedianFilteredImage)
      return adaptiveMedianFilteredImage

def normalizeImage(image):
      normalizedImage = np.single(image) / 255.0
      return normalizedImage

def take_a_piece(big_matrix,i,j,padding_size):
      x1 = i-padding_size
      x2 = i+padding_size
      x3 = j-padding_size
      x4 = j+padding_size
      piece = big_matrix[x1:x2+1,x3:x4+1]
      return piece

def calculateVariance(piece):
      mean = np.mean(piece)
      variances = np.mean( (piece - mean)**2 )
      totalVariance = np.mean(variances)
      return totalVariance

def draw_images(image1, image2):
      cv.namedWindow("image1", cv.WINDOW_NORMAL)
      cv.namedWindow("image2", cv.WINDOW_NORMAL)
      cv.imshow("image1", image1)
      cv.imshow("image2", image2)
      cv.waitKey(0)
      cv.destroyAllWindows()   

Applying Adaptive Mean Filter : 
In this part if we choose our variance of Noise 0.009 our filter is corrupting the image and it won't work.So it is need to be known the varianceOfNoise(sigmoidn^2).

In [5]:
filterKernelSupportSize = 5
varianceOfNoise = 0.001225
output_1_1 = adaptiveMeanFilter(gaussianNoisyImage,filterKernelSupportSize,varianceOfNoise)


KeyboardInterrupt: 

In [ ]:
print(output_1_1)

[[170 177 166 ... 156 150 119]
 [158 164 157 ... 175 149 131]
 [165 175 166 ... 166 163 128]
 ...
 [ 39  34  45 ... 106 104 103]
 [ 35  35  46 ... 110 109 107]
 [ 41  35  48 ...  97 114  87]]


In [ ]:
draw_images(output_1_1,gaussianNoisyImage)

Applying Other 2 Denoising Filters : Box Filter and Gaussian Filter

In [ ]:
# Opencv .blur() API - NORMALIZED box filter.
output_1_2 = cv.blur(gaussianNoisyImage, (5,5))

# Opencv .GaussianBlur() API - Gaussian filter.
output_1_3 = cv.GaussianBlur(gaussianNoisyImage, (5,5), 0)

In [ ]:
draw_images(output_1_2, gaussianNoisyImage)
draw_images(output_1_3,gaussianNoisyImage)

Evaluation Part : 

In [ ]:
cleanImage = cv.imread("cleanImage.jpg",0)
print(cleanImage)

[[161 158 159 ... 168 154 116]
 [159 158 159 ... 166 152 119]
 [159 159 160 ... 169 151 119]
 ...
 [ 20  22  27 ...  92  92  83]
 [ 20  25  31 ...  92  92  96]
 [ 23  20  33 ...  90  91  98]]


In [ ]:

psnr = cv.PSNR(cleanImage,gaussianNoisyImage)
psnr1 = cv.PSNR(cleanImage,output_1_1)
psnr2 = cv.PSNR(cleanImage,output_1_2)
psnr3 = cv.PSNR(cleanImage,output_1_3)

In [ ]:
print("PSNR Value of Clean - Box Filtered           : ",psnr2)
print("PSNR Value of Clean - Noised                 : ",psnr)
print("PSNR Value of Clean - Gaussian Filtered      : ",psnr3)
print("PSNR Value of Clean - Adaptive Mean Filtered : ",psnr1)

PSNR Value of Clean - Box Filtered           :  24.91291477512988
PSNR Value of Clean - Noised                 :  25.76271314264556
PSNR Value of Clean - Gaussian Filtered      :  26.111154216423053
PSNR Value of Clean - Adaptive Mean Filtered :  27.070123753297217


Q - 2 : Adaptive Median Filter 

In [ ]:
saltPepperNoiseImage = cv.imread("SaltPepperNoise_1.jpg",0)
print(saltPepperNoiseImage)

[[161 158 159 ... 168 154 116]
 [159 158 159 ... 166 152 119]
 [159 159 160 ... 169 151 119]
 ...
 [ 20  22  27 ...  92  92  83]
 [ 20  25  31 ...  92  92  96]
 [ 23  20  33 ...  90  91  98]]


Applying Adaptive Median Filter to Image : 

In [ ]:
output_2_1 = adaptiveMedianFilter(saltPepperNoiseImage)

Recall of EXERCISE - 3 : Our Center Weighted Median Filter 

Methods from Exercise 3 : 

In [ ]:
def centerWeightedmedianFilter(image,medianFilterSize,weight):
      m,n = image.shape
      paddingSize = medianFilterSize // 2
      new_matrix = borderReplicating(image,paddingSize)
      centerWeightedMedianFilteredImage = np.zeros( (m,n) )
      for i in range(paddingSize, paddingSize+m):
            for j in range(paddingSize, paddingSize+n):
                  piece = take_a_piece(new_matrix,i,j,paddingSize)
                  temp = applyingWeight(piece,paddingSize,weight)
                  temp = np.sort(temp)
                  medianValue = temp[ (medianFilterSize**2) // 2 ]
                  centerWeightedMedianFilteredImage[i-paddingSize, j-paddingSize]= medianValue
      centerWeightedMedianFilteredImage = np.uint8(np.round(centerWeightedMedianFilteredImage))
      return centerWeightedMedianFilteredImage

def applyingWeight(piece,paddingSize,weight):
      m,n = piece.shape
      listOfPieceValues = []
      for i in range(m):
            for j in range(n):
                  if i== paddingSize and j == paddingSize : # Which means it is center.
                        for k in range(weight):
                              listOfPieceValues.append(piece[i,j])
                  else:
                        listOfPieceValues.append(piece[i,j])
      return listOfPieceValues

OpenCV's Built In Median Filters : 3x3 - 5x5 - 7x7

In [ ]:
output_2_2 = cv.medianBlur(saltPepperNoiseImage, 3)
output_2_3 = cv.medianBlur(saltPepperNoiseImage, 5)
output_2_4 = cv.medianBlur(saltPepperNoiseImage, 7)

Applying Center Weighted Filters : 3x3 - 5x5 - 7x7

In [ ]:
output_2_5 = centerWeightedmedianFilter(saltPepperNoiseImage,3,3)
output_2_6 = centerWeightedmedianFilter(saltPepperNoiseImage,5,5)
output_2_7 = centerWeightedmedianFilter(saltPepperNoiseImage,7,7)

Evaluation Part : PSNR Values

In [ ]:
psnrForSAP = cv.PSNR(cleanImage,saltPepperNoiseImage)
psnr4 = cv.PSNR(cleanImage,output_2_1)
psnr5 = cv.PSNR(cleanImage,output_2_2)
psnr6 = cv.PSNR(cleanImage,output_2_3)
psnr7 = cv.PSNR(cleanImage,output_2_4)
psnr8 = cv.PSNR(cleanImage,output_2_5)
psnr9 = cv.PSNR(cleanImage,output_2_6)
psnr10 = cv.PSNR(cleanImage,output_2_7)

In [ ]:
print("PSNR Value of Clean - Our Adaptive Median Filter        : ",psnr4)
print("PSNR Value of Clean - Noised                            : ",psnrForSAP)
print("PSNR Value of Clean - OpenCV Median Filtered  3x3       : ",psnr5)
print("PSNR Value of Clean - OpenCV Median Filtered  5x5       : ",psnr6)
print("PSNR Value of Clean - OpenCV Median Filtered  7x7       : ",psnr7)
print("PSNR Value of Clean - Center Weighted Median Filter 3x3 : ",psnr8)
print("PSNR Value of Clean - Center Weighted Median Filter 5x5 : ",psnr9)
print("PSNR Value of Clean - Center Weighted Median Filter 7x7 : ",psnr10)

PSNR Value of Clean - Our Adaptive Median Filter        :  37.74197740239303
PSNR Value of Clean - Noised                            :  31.17935178645491
PSNR Value of Clean - OpenCV Median Filtered  3x3       :  33.868636704002576
PSNR Value of Clean - OpenCV Median Filtered  5x5       :  29.930072057093287
PSNR Value of Clean - OpenCV Median Filtered  7x7       :  27.800558085364404
PSNR Value of Clean - Center Weighted Median Filter 3x3 :  35.680976337467776
PSNR Value of Clean - Center Weighted Median Filter 5x5 :  31.25905908433834
PSNR Value of Clean - Center Weighted Median Filter 7x7 :  28.884604800240542


Applying All Filters to a Second Image : 

In [ ]:
saltAndPepperNoisedImage2 = cv.imread("SaltPepperNoise_2.jpg",0)
print(saltAndPepperNoisedImage2)

[[161 158 159 ... 168 154 116]
 [159 158 159 ... 166 152 119]
 [159 159 160 ... 169 151 119]
 ...
 [ 20  22  27 ...  91 255  80]
 [ 20  25  31 ...  91  92  97]
 [ 23  20  33 ...  89  91  98]]


In [ ]:
output_3_1 = adaptiveMedianFilter(saltAndPepperNoisedImage2)
output_3_2 = cv.medianBlur(saltAndPepperNoisedImage2, 3)
output_3_3 = cv.medianBlur(saltAndPepperNoisedImage2, 5)
output_3_4 = cv.medianBlur(saltAndPepperNoisedImage2, 7)
output_3_5 = centerWeightedmedianFilter(saltAndPepperNoisedImage2,3,3)
output_3_6 = centerWeightedmedianFilter(saltAndPepperNoisedImage2,5,5)
output_3_7 = centerWeightedmedianFilter(saltAndPepperNoisedImage2,7,7)


In [ ]:
psnrForSAP = cv.PSNR(cleanImage,saltAndPepperNoisedImage2)
psnr1 = cv.PSNR(cleanImage,output_3_1)
psnr2 = cv.PSNR(cleanImage,output_3_2)
psnr3 = cv.PSNR(cleanImage,output_3_3)
psnr4 = cv.PSNR(cleanImage,output_3_4)
psnr5 = cv.PSNR(cleanImage,output_3_5)
psnr6 = cv.PSNR(cleanImage,output_3_6)
psnr7 = cv.PSNR(cleanImage,output_3_7)

In [ ]:
print("PSNR Value of Clean - Our Adaptive Median Filter        : ",psnr1)
print("PSNR Value of Clean - Noised                            : ",psnrForSAP)
print("PSNR Value of Clean - OpenCV Median Filtered  3x3       : ",psnr2)
print("PSNR Value of Clean - OpenCV Median Filtered  5x5       : ",psnr3)
print("PSNR Value of Clean - OpenCV Median Filtered  7x7       : ",psnr4)
print("PSNR Value of Clean - Center Weighted Median Filter 3x3 : ",psnr5)
print("PSNR Value of Clean - Center Weighted Median Filter 5x5 : ",psnr6)
print("PSNR Value of Clean - Center Weighted Median Filter 7x7 : ",psnr7)

PSNR Value of Clean - Our Adaptive Median Filter        :  34.908456827452724
PSNR Value of Clean - Noised                            :  22.06821686946401
PSNR Value of Clean - OpenCV Median Filtered  3x3       :  33.509311680314184
PSNR Value of Clean - OpenCV Median Filtered  5x5       :  29.830583699377243
PSNR Value of Clean - OpenCV Median Filtered  7x7       :  27.76026993903325
PSNR Value of Clean - Center Weighted Median Filter 3x3 :  34.92309622265551
PSNR Value of Clean - Center Weighted Median Filter 5x5 :  31.06791469197246
PSNR Value of Clean - Center Weighted Median Filter 7x7 :  28.811044029226068
